# PyChe Tutorial: Run + Full Diagnostics

This notebook shows in-memory analysis from `res.mod` and `res.fis`, file-based diagnostics, and MPI workflows.

It includes:
- serial in-memory run
- MPI in-memory run with `mpi_subprocess=True`
- baseline/no-approx comparison
- diagnostic plots from `res.mod` and `res.fis`
- MPI test recipes and interpretation notes


In [1]:
from pyche import GCEModel, create_diagnostic_plots, read_outputs
from pyche.diagnostics import diagnostics_from_tables
import numpy as np
import matplotlib.pyplot as plt


## 1) In-memory run (serial)


In [3]:
m = GCEModel()
res = m.GCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=True,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
#    progress_style="single",
)
res.mod.shape, res.fis.shape


Progress [##############################] 100.00% (13700/13700)
GCE full translation run complete
ninputyield: 32
final gas: 495699.2887694819
final zeta: 0.006259913397521376
timing profile (s): total=6.157, interp=4.406, mpi_reduce=0.000, death=0.401, wind=1.083, output=0.024, other=0.243
total runtime (wall): 6.157 s


((13700, 39), (13700, 12))

## 1a) In-memory run (MPI from notebook)

Use `mpi_subprocess=True` when you want the result directly as a Python variable from a notebook cell.


In [4]:
m = GCEModel()
res = m.GCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    show_progress=True,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
    use_mpi=True,
    mpi_subprocess=True,
    mpi_subprocess_ranks=4,
    progress_style="single",

)
res.mod.shape, res.fis.shape


'Progress [##############################] 100.00% (13700/13700)'

GCE full translation run complete
MPI ranks: 4
ninputyield: 32
final gas: 495699.2887694819
final zeta: 0.006259913397521376
timing profile (s): total=3.492, interp=1.254, mpi_reduce=0.226, death=0.456, wind=1.230, output=0.026, other=0.300
total runtime (wall, max-rank): 3.492 s



((13700, 39), (13700, 12))

## 1b) Same run without approximations (baseline settings, serial)


In [6]:
m = GCEModel()
res_noapprox = m.GCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=True,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
    adaptive_timestep=False,
    interp_cache=False,
    interp_cache_guard=False,
    profile_timing=False,
    spalla_stride=1,
    spalla_inactive_threshold=0.0,
    spalla_lut=False,
)
res_noapprox.mod.shape, res_noapprox.fis.shape


Progress [##############################] 100.00% (13700/13700)
GCE full translation run complete
ninputyield: 32
final gas: 495745.9622989858
final zeta: 0.00625895498464687
total runtime (wall): 41.693 s


((13700, 39), (13700, 12))

## 1c) Same baseline settings in MPI (optional)


In [7]:
m = GCEModel()
res_noapprox_mpi = m.GCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    show_progress=True,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
    adaptive_timestep=False,
    interp_cache=False,
    interp_cache_guard=False,
    profile_timing=False,
    spalla_stride=1,
    spalla_inactive_threshold=0.0,
    spalla_lut=False,
    use_mpi=True,
    mpi_subprocess=True,
    mpi_subprocess_ranks=4,
)
res_noapprox_mpi.mod.shape, res_noapprox_mpi.fis.shape


'Progress [##############################] 100.00% (13700/13700)'

GCE full translation run complete
MPI ranks: 4
ninputyield: 32
final gas: 495745.9622989858
final zeta: 0.00625895498464687
total runtime (wall, max-rank): 30.998 s



((13700, 39), (13700, 12))

## Output column names


In [ ]:
print('mod columns:', res.mod_columns)
print('fis columns:', res.fis_columns)


## 2) Diagnostics from in-memory arrays


In [ ]:
diag = diagnostics_from_tables(res.mod, res.fis)
diag


In [ ]:
# Helper extractors for optimized run and no-approx run
mod_cols = {name: i for i, name in enumerate(res.mod_columns)}
fis_cols = {name: i for i, name in enumerate(res.fis_columns)}

def extract_series(r):
    time_f = r.fis[:, fis_cols['time']]
    sfr = r.fis[:, fis_cols['sfr']]
    allv = r.fis[:, fis_cols['all']]
    gas = r.fis[:, fis_cols['gas']]
    stars = r.fis[:, fis_cols['stars']]
    remn = r.fis[:, fis_cols['remn']]
    zeta = r.fis[:, fis_cols['zeta']]

    time_m = r.mod[:, mod_cols['time']]
    fe = r.mod[:, mod_cols['Fe']]
    h = r.mod[:, mod_cols['H']]
    o16 = r.mod[:, mod_cols['O16']]
    mg = r.mod[:, mod_cols['Mg']]

    eps = 1.0e-30
    FE_H_SUN_MASS = 56.0 * 10.0 ** (-4.50)
    O_FE_SUN_MASS = (16.0 / 56.0) * 10.0 ** (8.69 - 7.50)
    MG_FE_SUN_MASS = (24.305 / 56.0) * 10.0 ** (7.60 - 7.50)
    log_feh = np.log10(np.maximum(fe / np.maximum(h, eps), eps) / FE_H_SUN_MASS)
    log_o_fe = np.log10(np.maximum(o16 / np.maximum(fe, eps), eps) / O_FE_SUN_MASS)
    log_mg_fe = np.log10(np.maximum(mg / np.maximum(fe, eps), eps) / MG_FE_SUN_MASS)

    stars_delta = np.diff(stars, prepend=stars[0])
    stars_delta = np.maximum(stars_delta, 0.0)
    mdf_mask = np.isfinite(log_feh) & (log_feh >= -5.0) & (log_feh <= 1.0)

    return {
        'time_f': time_f, 'sfr': sfr, 'allv': allv, 'gas': gas, 'stars': stars, 'remn': remn, 'zeta': zeta,
        'time_m': time_m, 'log_feh': log_feh, 'log_o_fe': log_o_fe, 'log_mg_fe': log_mg_fe,
        'stars_delta': stars_delta, 'mdf_mask': mdf_mask,
    }

opt = extract_series(res)
base = extract_series(res_noapprox)

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(opt['time_f'], opt['sfr'], label='optimized')
ax[0].plot(base['time_f'], base['sfr'], '--', label='no_approx')
ax[0].set_title('SFR vs Time')
ax[0].legend(frameon=False)
ax[1].plot(opt['time_f'], opt['zeta'], label='optimized')
ax[1].plot(base['time_f'], base['zeta'], '--', label='no_approx')
ax[1].set_title('Zeta vs Time')
ax[1].legend(frameon=False)
plt.tight_layout()

fig, ax = plt.subplots(figsize=(7, 4))
for name in ['allv','gas','stars','remn']:
    ax.plot(opt['time_f'], opt[name], label=f'{name} opt')
    ax.plot(base['time_f'], base[name], '--', label=f'{name} base')
ax.set_title('Mass Budget Evolution')
ax.legend(frameon=False, ncols=2, fontsize=8)
plt.tight_layout()

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(opt['time_m'], opt['log_feh'], lw=1.4, label='optimized')
ax.plot(base['time_m'], base['log_feh'], '--', lw=1.2, label='no_approx')
ax.set_title('[Fe/H] vs Time')
ax.set_ylim(-7,0.8)
ax.legend(frameon=False)
plt.tight_layout()

mask_opt = np.isfinite(opt['log_feh'])
mask_base = np.isfinite(base['log_feh'])
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(opt['log_feh'][mask_opt], opt['log_o_fe'][mask_opt], label='[O/Fe] opt', lw=1.3)
ax.plot(base['log_feh'][mask_base], base['log_o_fe'][mask_base], '--', label='[O/Fe] base', lw=1.1)
ax.plot(opt['log_feh'][mask_opt], opt['log_mg_fe'][mask_opt], label='[Mg/Fe] opt', lw=1.3)
ax.plot(base['log_feh'][mask_base], base['log_mg_fe'][mask_base], '--', label='[Mg/Fe] base', lw=1.1)
ax.set_title('Abundance Tracks')
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('[X/Fe]')
ax.set_xlim(-5,1)
ax.set_ylim(-0.1,0.8)
ax.legend(frameon=False, fontsize=8)
plt.tight_layout()

bins = np.linspace(-5, 1, 61)
w_opt = opt['stars_delta'][opt['mdf_mask']].copy()
w_base = base['stars_delta'][base['mdf_mask']].copy()
if np.sum(w_opt) > 0:
    w_opt /= np.sum(w_opt)
if np.sum(w_base) > 0:
    w_base /= np.sum(w_base)
fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(opt['log_feh'][opt['mdf_mask']], bins=bins, weights=w_opt, alpha=0.5, label='optimized')
ax.hist(base['log_feh'][base['mdf_mask']], bins=bins, weights=w_base, alpha=0.5, label='no_approx')
ax.set_title('MDF [Fe/H]')
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('Normalized Stellar-Mass Fraction')
ax.legend(frameon=False)
plt.tight_layout()


## 3) Save a run, load it back, and create plot files


In [ ]:
out_dir = 'RISULTATI_PYCHE_NOTEBOOK'
m.GCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=False,
    output_dir=out_dir,
    output_mode='dataframe',
    df_binary_format='pickle',
    write_output=True,
    return_results=False,
)
payload = read_outputs(out_dir, prefer='dataframe', binary_format='pickle')
payload['mod'].shape, payload['fis'].shape, payload['format']

paths = create_diagnostic_plots(out_dir, prefer='dataframe', binary_format='pickle')
paths


## 4) MPI test recipes (4 ranks)

### A) Recommended production run (shell)
```bash
mpiexec -n 4 python -c "from pyche import GCEModel; m=GCEModel(); m.GCE(13700,3000.0,50.0,0.3,0.0,10000,10000,use_mpi=True,show_progress=False,backend='auto',output_dir='RISULTATI_MPI4',output_mode='dataframe',df_binary_format='pickle')"
```

### B) Cython backend (compile first)
```bash
pip install cython
python setup.py build_ext --inplace
mpiexec -n 4 python -c "from pyche import GCEModel; m=GCEModel(); m.GCE(13700,3000.0,50.0,0.3,0.0,10000,10000,use_mpi=True,show_progress=False,backend='cython',output_dir='RISULTATI_MPI4_CY',output_mode='dataframe',df_binary_format='pickle')"
```

### MPI test notes
- For fair speed tests, set `show_progress=False` (progress output adds overhead).
- Compare serial vs MPI with identical physics flags (`adaptive_timestep`, cache/guard, LUT flags).
- Use at least 2 runs and compare wall time from the printed `total runtime (wall, max-rank)`.
- If MPI output appears buffered in notebook environments, prefer `mpi_subprocess=True` or run from a shell.
- If `mpi4py` warns about MPI mismatch, rebuild `mpi4py` with the same MPI implementation used by `mpiexec`.
